In [2]:
import pandas as pd
import numpy as np
import optuna

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error

## Model parameters

In [3]:
# TODO: don't forget device='cuda'
xgb_parameters = {
    'max_leaves': 40,
    'max_depth': 281,
    'learning_rate': 0.08792944959024201,
    'n_estimators': 632,
    'subsample': 0.6900703019224456,
    'colsample_bytree': 0.767812706804894,
    'device': 'cuda',
}

lgbm_parameters = {
    'num_leaves': 66,
    'max_depth': 449,
    'learning_rate': 0.03502441310895256,
    'n_estimators': 820,
    'subsample': 0.592592101509771,
    'colsample_bytree': 0.6862792428425046
}

sgd_parameters = {
    'penalty': None,
    'alpha': 4.223601774273774,
    'l1_ratio': 0.8940821225850496
}

bayes_ridge_parameters = {
    'alpha_1': 0.012972867153230679,
    'alpha_2': 0.03937728242649084,
    'lambda_1': 5.3000736672220825e-06,
    'lambda_2': 0.07761015011781103
}

cat_parameters = {
    'learning_rate': 0.029247253288660423,
    'depth': 9,
    'l2_leaf_reg': 6.715144149188259,
    'random_strength': 0.29075067665837756,
    'bagging_temperature': 0.3097108651010562,
    'task_type': 'GPU',
}

## Data and evaluation

In [4]:
df = pd.read_csv('data/train_pp.csv')
test_df = pd.read_csv('data/test_pp.csv')

X = df.drop(['accident_risk', 'id'], axis=1)
y = df['accident_risk']

In [5]:
def submission_generator(trained_model):
    test_df_preprocessed = test_df.drop('id', axis=1)
    return pd.concat([test_df['id'], pd.Series(trained_model.predict(test_df_preprocessed))], axis=1).rename({0: 'accident_risk'}, axis=1)

## Model prepration

In [6]:
xgb_model = XGBRegressor(**xgb_parameters)
lgbm_model = LGBMRegressor(**lgbm_parameters)
sgd_model = SGDRegressor(**sgd_parameters)
bayes_model = BayesianRidge(**bayes_ridge_parameters)
cat_model = CatBoostRegressor(**cat_parameters)

In [7]:
estimators = [
    ('xgb', xgb_model),
    ('lgbm', lgbm_model),
    ('sgd', sgd_model),
    ('bayes', bayes_model),
    ('cat', cat_model),
]

## Default ridge cv

In [8]:
kf = KFold(5, shuffle=True)

rmses = []

for train_index, test_index in kf.split(X):
    # Use default RidgeCV model with out passthrough
    stacking_regressor = StackingRegressor(estimators)
    
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    stacking_regressor.fit(X_train, y_train)
    y_pred = stacking_regressor.predict(X_test)

    rmse = root_mean_squared_error(y_pred, y_test)
    rmses.append(rmse)
    print('---' * 10)
    print(rmse)
    print('---' * 10)

np.mean(rmses) 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352461
0:	learn: 0.1621487	total: 11.1ms	remaining: 11.1s
1:	learn: 0.1581092	total: 15.4ms	remaining: 7.71s
2:	learn: 0.1542029	total: 19.2ms	remaining: 6.39s
3:	learn: 0.1504334	total: 22.6ms	remaining: 5.62s
4:	learn: 0.1467757	total: 25.8ms	remaining: 5.14s
5:	learn: 0.1432439	total: 30.4ms	remaining: 5.03s
6:	learn: 0.1398221	total: 33.8ms	remaining: 4.79s
7:	learn: 0.1365164	total: 37.5ms	remaining: 4.65s
8:	learn: 0.1333197	total: 40.8ms	remaining: 4.49s
9:	learn: 0.1302301	total: 44.2ms	remaining: 4.38s
10:	learn: 0.1272574	total: 48.7ms	remaining: 4.38s
11:	learn: 0.1243802	total: 52.5

/home/woxane/anaconda3/lib/python3.10/site-packages/xgboost/core.py:774: UserWarning: [22:42:06] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 427
[LightGBM] [Info] Number of data points in the train set: 331362, number of used features: 17
[LightGBM] [Info] Start training from score 0.352647
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 331362, number of used features: 17
[LightGBM] [Info] Start training from score 0.352425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002594 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

np.float64(0.05596557181308333)

In [9]:
stacking_regressor = StackingRegressor(estimators)
stacking_regressor.fit(X, y)

submission = submission_generator(stacking_regressor)
submission.to_csv('stacking_meta_optuna.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 517754, number of used features: 17
[LightGBM] [Info] Start training from score 0.352377
0:	learn: 0.1622390	total: 3.89ms	remaining: 3.89s
1:	learn: 0.1581958	total: 7.54ms	remaining: 3.77s
2:	learn: 0.1542864	total: 11.3ms	remaining: 3.74s
3:	learn: 0.1505057	total: 15ms	remaining: 3.72s
4:	learn: 0.1468524	total: 18.6ms	remaining: 3.7s
5:	learn: 0.1433061	total: 22.3ms	remaining: 3.69s
6:	learn: 0.1398870	total: 25.9ms	remaining: 3.67s
7:	learn: 0.1365796	total: 29.4ms	remaining: 3.65s
8:	learn: 0.1333802	total: 33.1ms	remaining: 3.65s
9:	learn: 0.1302913	total: 36.7ms	remaining: 3.64s
10:	learn: 0.1273069	total: 40.3ms	remaining: 3.63s
11:	learn: 0.1244367	total: 43.9ms	